In [23]:
import requests
import json
import pandas as pd
from getpass import getpass
import dotenv
from dotenv import load_dotenv
import os 
from pymongo import MongoClient

In [6]:
token = getpass()

In [7]:
token

'fsq3FKRSnl7QmegrfIxRzRsClESef0ln3jDAT5Mq9KRpi1c='

In [ ]:
# I am going to request how many of the following locations I have in sydney, and where is located.

# Residential or commercial areas to attract players
# Parks or recreational areas for outdoor activities
# Schools or universities for student athletes and recreational players
# Health clubs or gyms for fitness enthusiasts
# Social Clubs 
# Bars or cafes for post-game meals and socializing
# Convenient public transportation options
# physiotherapy 
# Tennis store
# Other sports facilities or fields to attract sports enthusiasts and teams.

In [45]:
# I create a dictionary with all the locations and its code associated in 4sq.
dict_of_places2 = {'bar':13003,
                   'physiotherapy':15026,
                   'social_club':12097,
                   'tennis_store':17130,
                   'residential':12094,
                   'park': 16032,
                   'university': 12013,
                   'gym':18021,
                   'parking':19020,
                   'public_transport':19054,
                   'tennis_court':18047
                    }
dict_of_places = {'bar':13003}

In [46]:
# I create two functions, one to request to 4sq and the other one to insert the result to mongo.

def request_4sq (key,dictionary):
    ''' This functions takes the key and the dictionary
        returns the respons of the request to 4sq based on the key we passed. '''

    center_point = '-33.8149474%2C150.9999835'
    radius = 30000
    headers = {"accept": "application/json",
                    "Authorization": token }
    category_code = dictionary[key]
    limit = 30
    url = f"https://api.foursquare.com/v3/places/search?ll={center_point}&radius={radius}&categories={category_code}&fields=name%2Cgeocodes&limit={limit}"

    response = requests.get(url, headers=headers).json()
    return response['results']

def send_to_mongo (key,response):
    '''This function takes the key and the response from 4sq
        insert the result in mongo and returns a message confirming.'''
    client = MongoClient("localhost:27017")
    db = client['geospatial']
    collection = db[key]
    collection.insert_many(response)

In [47]:
for key in dict_of_places.keys():
    response = (request_4sq(key,dict_of_places))
    send_to_mongo(key,response)
    




An error occurred: 'NoneType' object is not iterable


In [ ]:
# I create two functions, one to request to 4sq and the other one to insert the result to mongo.

def request_4sq (key,dictionary):
    ''' This functions takes the key and the dictionary
        returns the respons of the request to 4sq based on the key we passed. '''

    center_point = '-33.8149474%2C150.9999835'
    radius = 30000
    headers = {"accept": "application/json",
                    "Authorization": token }
    category_code = dictionary[key]
    limit = 30
    url = f"https://api.foursquare.com/v3/places/search?ll={center_point}&radius={radius}&categories={category_code}&fields=name%2Cgeocodes&limit={limit}"

    response = requests.get(url, headers=headers)
    return response

def send_to_mongo (key,response):
    '''This function takes the key and the response from 4sq
        insert the result in mongo and returns a message confirming.'''
    try:    
        if response.status_code == 200:
            # go ahead with the rest of your code
            response = list(response.json()['results'])
            client = MongoClient("localhost:27017")
            db = client['geospatial']
            collection = db[key]
            successfully = 0
            duplicated = 0
            # Check if the element exists and insert it.
            for element in response:
                if len(list(collection.find_one(element['name']))) > 0:
                    collection.insert_one(element)
                    successfully += 1
                    
                else:
                    duplicated +=1 
            return print(f'You have inserted {successfully} documents, and you had {duplicated} already in the db.')
            # do something with the data
        else:
            print("Error response from API:", response.text)
            raise Exception("Received non-200 response")
    except requests.exceptions.RequestException as e:
        # handle exceptions raised by the requests library
        print("Request failed:", e)
    except Exception as e:
        # handle any other exceptions
        print("An error occurred:", e)